In [1]:
"""
  A manager that facilitates reading and writing files to GCP Storage
"""
import logging
import os
import subprocess
from io import BytesIO
from typing import List, Dict, Callable, Tuple, Union
from mypy_extensions import TypedDict

from google.cloud import storage # type: ignore

def get_path_prefix(root_dir: str, relative_path: str) -> str:
    prefix = ''
    if root_dir:
        prefix = root_dir.rstrip('/') + '/'

    if relative_path and relative_path != '/':
        prefix = prefix + relative_path.strip('/') + '/'

    return prefix

class PathNode(TypedDict, total=False):
    name: str
    type: str
    size: float
        
class GCPStorageManager(object):

    def __init__(self, storage_details: Dict, verbose: bool) -> None:
        self._bucket_name = storage_details.get('bucket')
        self._root_dir = storage_details.get('root')
        self.client = storage.Client()
        self.verbose = verbose

    def _abs_path(self, rel_path: str) -> str:
        if not self._root_dir:
            return rel_path

        return os.path.join(self._root_dir, rel_path)

    def _build_current_url(self) -> str:
        if self._root_dir:
            return "https://console.cloud.google.com/storage/browser/" + self._bucket_name + "/" + self._root_dir
        else:
            return "https://console.cloud.google.com/storage/browser/" + self._bucket_name

    def get_root_dir(self) -> str:
        return self._root_dir

    def get_storage_details(self) -> Dict:
        return {
            'provider': file_utils.ProviderList.GCP_STORAGE,
            'bucket': self._bucket_name,
            'root': self._root_dir
        }

    def get_sync_login_command(self, env_vars: Dict) -> List[str]:
        return None

    def get_sync_url(self, path: str) -> str:
        if not path or path == '/':
            abs_path = self._root_dir
        else:
            abs_path = self._abs_path(path)
        return f'gs://{self._bucket_name}/{abs_path}'

    def get_sync_command(self, src_dir: str, remote_path: str) -> Callable:

        def sync_call() -> Tuple[int, str]:
            cmd = ['gsutil', 'rsync', '-r', src_dir, self.get_sync_url(remote_path)]
            p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout, stderr = p.communicate()
            err_code = 0
            stderr_val = ''

            if p.returncode != 0:
                logging.error('Copy response is: {}'.format(stderr.decode('utf-8')))
                stderr_val = stderr.decode('utf-8')
                err_code = p.returncode

            return err_code, stderr_val

        return sync_call

    def rm_file(self, relative_path: str) -> None:
        bucket = self.client.bucket(self._bucket_name)
        path = self._abs_path(relative_path)
        blob = bucket.blob(path)
        logging.info('Deleting file at ' + path)
        blob.delete()

    def rm_dir(self, relative_path: str) -> None:
        bucket = self.client.bucket(self._bucket_name)
        prefix = get_path_prefix(self._root_dir, relative_path)
        blob = bucket.blob(prefix)
        logging.info('Deleting files at ' + prefix)
        blob.delete()

    def read_content(self, path: str, throw_exception: bool, read_range: str = None, streaming: bool = False) -> bytes:
        try:
            path = self._abs_path(path)
            bucket = self.client.bucket(self._bucket_name)
            blob = bucket.blob(path)
            result = blob.download_as_bytes()

            if self.verbose:
                logging.info(f"Downloading content from {self._build_current_url()}/{path}")

            return result
        except Exception as e:
            raise

    #def gen_presigned_url(self, path: str) -> str:
    #    # dask and other frameworks explicit s3 link rather than a byte stream or contet (e.g. s3://bucket/foo.csv)
    #    return self._s3.generate_presigned_url('get_object',
    #                                           Params={'Bucket': self._bucket_name, 'Key': self._abs_path(path)})

    # Checks to see if the job directory exists.  No side-effects.
    def check_dir_exists(self, path: str) -> bool:
        # Create the prefix for this particular job.
        exists = False
        prefix = get_path_prefix(self._root_dir, path)
        blobs = list(self.client.list_blobs(
            self._bucket_name, prefix=prefix
        ))
        if len(blobs) > 0:
            exists = True

        return exists

    def _download_content(self, remote_path: str) -> bytes:
        remote_path = self._abs_path(remote_path)
        bucket = self.client.bucket(self._bucket_name)
        blob = bucket.blob(remote_path)
        if self.verbose:
            logging.info(f"Downloading content from {self._build_current_url()}/{remote_path}")

        fileobj = BytesIO()
        blob.download_to_file(fileobj)
        return fileobj.getvalue()

    def download_file(self, remote_path: str, file_name: str) -> None:
        # Move references to large data items across folders
        remote_path = self._abs_path(remote_path)
        bucket = self.client.bucket(self._bucket_name)
        blob = bucket.blob(remote_path)
        if self.verbose:
            logging.info(f"Downloading file {self._build_current_url()}/{remote_path} to {file_name}")

        dirname = os.path.dirname(file_name)

        if dirname:
            # Only create a directory if it's not ''
            if not os.path.exists(dirname):
                os.makedirs(dirname)

        blob.download_to_filename(file_name)

    def download_and_unzip(self, remote_path: str, local_dir: str) -> None:
        zip_bytes = self._download_content(remote_path)
        file_utils.unzip_into_dir(zip_bytes, local_dir)

    def download_dir(self, remote_path: str, local_path: str) -> int:
        nFiles = 0
        if self.verbose:
            logging.info("Downloading folder: " + remote_path + " to " + local_path)

        prefix = file_utils.get_path_prefix(self._root_dir, remote_path)
        for blob in self.client.list_blobs(self._bucket_name, prefix=prefix):
            if blob.name.endswith('/'):
                continue

            rel_path = os.path.relpath(blob.name, prefix)
            dest_pathname = os.path.join(local_path, rel_path)

            if not os.path.exists(os.path.dirname(dest_pathname)):
                os.makedirs(os.path.dirname(dest_pathname))

            if self.verbose:
                logging.info(f"Downloading file {blob.name} to {dest_pathname}")

            blob.download_to_filename(dest_pathname)  # Download
            nFiles += 1

        return nFiles

    def upload_content(self, content: bytes, file_name: str) -> None:
        # Uploads file content to a specific filename location
        bucket = self.client.bucket(self._bucket_name)
        if self.verbose:
            logging.info(f"Uploading content to {self._build_current_url()}/{file_name}")

        blob = bucket.blob(self._abs_path(file_name))
        blob.upload_from_file(BytesIO(content))

    def list_directory(self, path: str, with_size: bool = False) -> Dict:
        prefix = get_path_prefix(self._root_dir, path)
        bucket = self.client.bucket(self._bucket_name)
        blobs = bucket.list_blobs(prefix=prefix)
        nodes = []
        for blob in blobs:
            if blob.name.endswith('/'):
                nodes.append(PathNode(name=os.path.basename(blob.name[:-1]), type='folder'))
            else:
                nodes.append(PathNode(name=os.path.basename(blob.name), type='file'))

        return {
            'nodes': nodes
        }


In [2]:
import os

from dotenv import load_dotenv
load_dotenv(verbose=True)

print(os.environ.get('GOOGLE_APPLICATION_CREDENTIALS'))

/Users/warren/sweat-equity-ventures/bespoke/flowhub_gcp_credentials.json


In [3]:
manager = GCPStorageManager({
    'bucket': 'partnerships-data-reporting',
    'root': ''
}, verbose=True)

In [8]:
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

In [12]:
import json
import pandas as pd
directory_contents = manager.list_directory('')['nodes']

In [13]:
company_ids_set = set([])

for file_metadata in directory_contents:
    file_name = file_metadata['name']
    # Company locations file case.
    if '_locations.csv' in file_name:
        company_id = file_name.replace('_locations.csv', '')
        company_ids_set.add(company_id)

company_ids_count = len(list(company_ids_set))
print(f'There are {company_ids_count} companies in the dataset')

There are 100 companies in the dataset


In [14]:
company_location_ids_set = set([])

for company_id in company_ids_set:
    locations_csv_string = manager.read_content(f'{company_id}/{company_id}_locations.csv', throw_exception=True)
    locations_io = StringIO(locations_csv_string.decode("utf-8"))
    locations_dataframe = pd.read_csv(locations_io)
    location_ids = locations_dataframe['locationId'].tolist()
    for location_id in location_ids:
        company_location_ids_set.add((company_id, location_id))

company_location_ids_count = len(list(company_location_ids_set))
print(f'There are {company_location_ids_count} company locations in the dataset')

There are 178 company locations in the dataset


In [15]:
# (company_id, location_id, location_sales_dataframe)
company_location_dataframes = []

for index, company_location_tuple in enumerate(company_location_ids_set):
    company_id, company_location_id = company_location_tuple
    print(f'[{index + 1}] Downloading sales dataframe for location ({company_id}, {company_location_id})')

    try:
        location_sales_csv_string = manager.read_content(f'{company_id}/{company_location_id}/{company_location_id}_sales.csv', throw_exception=True)
        location_sales_io = StringIO(location_sales_csv_string.decode("utf-8"))
        location_sales_dataframe = pd.read_csv(location_sales_io)
        company_location_dataframes.append((company_id, company_location_id, location_sales_dataframe))
    except:
        print(f'[{index + 1}] An exception occurred for location ({company_id}, {company_location_id})')

company_location_dataframes_count = len(list(company_location_dataframes))
print(f'There are {company_location_dataframes_count} company location dataframes in the dataset')

[1] Downloading sales dataframe for location (pTYqYSwRCuxLKKzME, QQvD4yk7CAtH2MDwa)
[2] Downloading sales dataframe for location (Q2ctDDsxjy23AoQLb, QzxtSrfS8dh3qWsWm)
[3] Downloading sales dataframe for location (T3nGGs9v9BkBJkLAJ, TpetsriCnhipdAvdb)
[4] Downloading sales dataframe for location (SPYsqAwk5XEjp9ZwX, wvkqmogsjjjfSSocY)
[5] Downloading sales dataframe for location (rPXmTTcY4LXmgmTcG, PYKgsLK9FTwcSnmaT)
[6] Downloading sales dataframe for location (6YRM2orTfSySDKhLL, R5EZkhoLBF9e7qxmF)
[7] Downloading sales dataframe for location (E2ZWysfDHW2pqB7JW, icLHyCEKBqp4b4sGK)
[8] Downloading sales dataframe for location (dJg8Bmr3FDZhipSJe, rZCY6mc4x8y7Fepu6)
[9] Downloading sales dataframe for location (j7MWhCyfRAgZstQdE, WPz3hFKPNePY6kFyT)
[10] Downloading sales dataframe for location (GNPWTqYqZsfcAA6Jb, tjPZFyNwNnYMc8jcE)
[11] Downloading sales dataframe for location (KkyhzzHqgJDFWPM9n, Ceoy5YkSujnhNt5Zn)
[12] Downloading sales dataframe for location (9jrcBWKRkf9wgQ6mr, 6EAiLXwC

[97] Downloading sales dataframe for location (rgNDsz626CTLnd53Y, NsTkwrKuNqSPpjxJn)
[98] Downloading sales dataframe for location (exFwNR5F4o5DGDKRW, AmkkHfRm6RjhEJGrK)
[99] Downloading sales dataframe for location (j7MWhCyfRAgZstQdE, Xa54Pov4FpuTxizDd)
[100] Downloading sales dataframe for location (D2JsangYtSFya4nDT, G4yyguo4R7wGp3NR8)
[101] Downloading sales dataframe for location (QtFmR4Qgdk2fcYQ2h, p4uoisdtyfqeYk4GX)
[102] Downloading sales dataframe for location (jfD4EGaLur6xsEHbQ, Wjgbcenow3oSALsgJ)
[103] Downloading sales dataframe for location (E2ZWysfDHW2pqB7JW, aYt5FfpYq2GnTagfC)
[104] Downloading sales dataframe for location (kEYekxChnHJ5FRAc4, pwJ65xWrRyZ4QFJGP)
[105] Downloading sales dataframe for location (FuDLDjBXzkEPPQxZD, TgLFQQcK4NiY8unCE)
[106] Downloading sales dataframe for location (KypyS8kNFhSPZxv7q, RJxDzpLGARNoej3x9)
[107] Downloading sales dataframe for location (rrCXhvarvXo7P2fGo, g63EjGYZsjHPqyPL4)
[108] Downloading sales dataframe for location (QZaMHjsdR

In [16]:
for company_id, location_id, location_sales_dataframe in company_location_dataframes:
    # TODO: do data science stuff on these dataframes.
    print(company_id, location_id, location_sales_dataframe.shape)

pTYqYSwRCuxLKKzME QQvD4yk7CAtH2MDwa (50, 14)
Q2ctDDsxjy23AoQLb QzxtSrfS8dh3qWsWm (16478, 14)
T3nGGs9v9BkBJkLAJ TpetsriCnhipdAvdb (21752, 14)
SPYsqAwk5XEjp9ZwX wvkqmogsjjjfSSocY (126292, 14)
rPXmTTcY4LXmgmTcG PYKgsLK9FTwcSnmaT (11445, 14)
6YRM2orTfSySDKhLL R5EZkhoLBF9e7qxmF (21429, 14)
E2ZWysfDHW2pqB7JW icLHyCEKBqp4b4sGK (38178, 14)
dJg8Bmr3FDZhipSJe rZCY6mc4x8y7Fepu6 (99506, 14)
j7MWhCyfRAgZstQdE WPz3hFKPNePY6kFyT (1968, 14)
GNPWTqYqZsfcAA6Jb tjPZFyNwNnYMc8jcE (55297, 14)
KkyhzzHqgJDFWPM9n Ceoy5YkSujnhNt5Zn (0, 14)
9jrcBWKRkf9wgQ6mr 6EAiLXwCs9wfXHbgG (86911, 14)
ySB2RKAtEDYKERK9J 2t3guxpfanCWqycA6 (26957, 14)
hPzYzThkKMuQrF9Q2 zKiH8jXK5S7cw5j5N (16898, 14)
KkyhzzHqgJDFWPM9n muYwpddLcAc6gTYYS (0, 14)
FhSceFrGthqNntkEE nkTZx6zDkxzBitF2D (253055, 14)
rgNDsz626CTLnd53Y HJho8PdduZazwxzet (15942, 14)
iJscrRfw6uYjEhEKo m4qw2bLpeNRpkwSog (74041, 14)
wiyY5tNikq6WRPQ6E q5ibpyhJiEuBwYCJv (331585, 14)
iJscrRfw6uYjEhEKo KrCPtJaQmpa3cjXXN (67631, 14)
KypyS8kNFhSPZxv7q w2nkNYHgjfaWNPiCZ (0, 14)
7R5MB